In [25]:
import traceback
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
import os
import cv2
from keras.models import load_model
from sklearn.utils import shuffle
import seaborn as sns
from scipy.io import loadmat
import pandas as pd
import numpy as np
from random import shuffle

import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
%matplotlib inline

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Data Preprocessing

In [27]:
y_translation = {
    0: 'Male',
    1: 'Female',
}
X_train, X_test, y_train, y_test = read_UTK(UTK_path)
classes = 2
inputShape = (200, 200, 1)

# Keras Neural Network Model Implementation

In [3]:
class Keras_NN:
    def __init__(self,X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train
        
    def train_model(self):
        IMG_SIZE = 48
        model = Sequential()
        model.add(Conv2D(64, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(7, activation = 'softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(self.X_train, self.Y_train, batch_size=len(self.X_train)//20, epochs=20, verbose=1)
        
        return model
    
    def load_model(self):
        model = load_model('keras_on_fer')
        return model
    
    def save_model(self, model, name):
        model.save(name)

# WebCam Functions


In [35]:
def display_emotions(model_emotion,model_gender, emotion_faces,gender_faces, frame, bboxes):
    '''
    Fetches the emotions by using model prediction and adds 
    them to the output frame
    '''
    emotion_translation = {
        0: 'Angry',
        1: 'Disgust',
        2: 'Fear',
        3: 'Happy',
        4: 'Sad',
        5: 'Surprise',
        6: 'Neutral'
    }
    
    gender_translation = {
        0: ' Man',
        1: ' Woman',
    }
    outputFrame = frame.copy()
    emotions = model_emotion.predict(emotion_faces)
    gender = model_gender.predict(gender_faces)
    for i in range(len(bboxes)):
        emotion = emotion_translation[np.argmax(emotions[i])]
        gender = gender_translation[np.argmax(gender[i])]
        result = emotion + gender
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(outputFrame, result,
                    (bboxes[i][0] - 1, bboxes[i][1] - 1), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(outputFrame, (bboxes[i][0], bboxes[i][1]), (
            bboxes[i][2], bboxes[i][3]), (0, 255, 0), int(round(outputFrame.shape[0]/150)), 8)
    return outputFrame

In [36]:
def get_webcam_feed(model_emotion, model_gender):
    '''
    Initializes webcam video for live classification
    Extracts faces from the frame
    Outputs the frame with predicted emotions to a window 
    '''
    cap = cv2.VideoCapture(0)
    hasFrame, frame = cap.read()
    # TODO : Handle cases for face going outside the frame
    frame_count = 0
    tt_opencvDnn = 0
    while(1):
        try:
            hasFrame, frame = cap.read()
            if not hasFrame:
                break
            frame_count += 1
            outputFrame = frame

            # Gives a list of gray-scale images in webcam feed
            emotion_faces, bboxes = face_reduction(frame, (48, 48))
            gender_faces, bboxes = face_reduction(frame, (200, 200))
            emotion_faces = np.array(emotion_faces)
            gender_faces = np.array(gender_faces)
            if emotion_faces.shape[0] != 0:
                #                 print(videoFaces.shape)
                emotion_faces = emotion_faces.reshape(
                    (emotion_faces.shape[0], emotion_faces.shape[1], emotion_faces.shape[2], 1))
                gender_faces = gender_faces.reshape(
                    (gender_faces.shape[0], gender_faces.shape[1], gender_faces.shape[2], 1))
                outputFrame = display_emotions(
                    model_emotion, model_gender, emotion_faces, gender_faces, frame, bboxes)
            cv2.imshow("frame", outputFrame)
            k = cv2.waitKey(10)

            # quits window when escape key is pressed
            if k == 27:
                break
        except Exception as e:
            print("Exception is ")
            print(e)
            traceback.print_tb(e.__traceback__)
            break
    cv2.destroyAllWindows()
    cap.release()

# Face Operations

In [37]:
def detectFaceOpenCVDnn(net, frame,image_size):
    result = []
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], False, False)
    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    conf_threshold = 0.7
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            if x1 > frameWidth or x1 < 0 or x2 > frameWidth or x2 < 0 or y1 < 0 or y1 > frameHeight or y2 < 0 or y2 > frameHeight:
                continue
            else:
                grayOpenDnn = gray = cv2.cvtColor(frameOpencvDnn, cv2.COLOR_BGR2GRAY)
                croppedOpenDnn = cv2.resize(gray[y1:y2,x1:x2], image_size) 
                result.append(croppedOpenDnn)
    return result, bboxes

In [38]:
def face_reduction(image, image_size):
    # OpenCV DNN supports 2 networks.
    # 1. FP16 version of the original caffe implementation ( 5.4 MB )
    # 2. 8 bit Quantized version using Tensorflow h( 2.7 MB )
#     print("printing image")
#     print(image)
    DNN = "TF"
    if DNN == "CAFFE":
        modelFile = "models/res10_300x300_ssd_iter_140000_fp16.caffemodel"
        configFile = "models/deploy.prototxt"
        net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
    else:
        modelFile = "models/opencv_face_detector_uint8.pb"
        configFile = "models/opencv_face_detector.pbtxt"
        net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

    conf_threshold = 0.7
    outOpencvDnn, bboxes = detectFaceOpenCVDnn(net,image,image_size)
    return outOpencvDnn, bboxes

In [39]:
model_NN = Keras_NN([],[])

In [40]:
emotion_model = model_NN.load_model()
gender_model = load_model('gender_classifier_model')

/Users/visheshhemnani/anaconda3/envs/ml_proj/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [ ]:
get_webcam_feed(emotion_model,gender_model)